In [22]:
import numpy as np
from typing import Self
from copy import deepcopy
from collections import Counter
import time

In [2]:
class SudokuBoard:
    def __init__(self):
        self._board = np.matrix(np.zeros((9,9),np.int64))
        
    def __str__(self):
        rows = []
        rows.append('#+------+-------+------+')
        for i in range(9):
            row = []
            for e in self._board[i, :].A1: 
                if e == 0:
                    row.append(' ')
                else:
                    row.append(str(e))
                    
            row.insert(3, '|')
            row.insert(7, '|')
            rows.append('#|'+' '.join(row)+'|')

            if i == 2 or i == 5 or i == 8:
                rows.append('#+------+-------+------+')

        return '\n'.join(rows)
    
    def at(self, row, col):
        return self._board[row, col]
    
    def as_matrix(self):
        return self._board
    
    def set(self, row, col, value):
        if row >= 0 and row < 9 and col >= 0 and col < 9:
            self._board[row, col] = value
            return True
        return False
    
    def set_all(self,values):
        for value in values:
            self.set(value[0], value[1], value[2])
    
    def is_solved(self):        
        rows_ok, cols_ok, squares_ok = self.solution_status()
        return rows_ok.all() and cols_ok.all() and squares_ok.all()
    
    def solution_status(self):
        rows_ok = np.apply_along_axis(
            SudokuBoard.check_1D, 
            axis=1, 
            arr=np.asarray(self._board)
        ).flatten()
        
        cols_ok = np.apply_along_axis(
            SudokuBoard.check_1D,
            axis=0,
            arr=np.asarray(self._board) 
        ).flatten()
        
        squares_ok = SudokuBoard.check_squares(self._board)
        
        return rows_ok, cols_ok, squares_ok
    
    @staticmethod
    def check_1D(row_or_col):
        numbers = set(list(range(1, 10)))
        return set(row_or_col) == numbers
    
    @staticmethod
    def check_squares(board):
        numbers = set(list(range(1, 10)))
        squares_ok = np.empty(9)
        for square in range(9):
            rows = range(3 * int(square/3), 3 * (1 + int(square/3)))
            cols = range(3 * (square % 3), 3 * (1 + (square % 3)))
            sub_board = board[rows, :][:, cols]
            squares_ok[square] = set(sub_board.A1) == numbers
        return squares_ok.astype(bool)

In [24]:
class SudokuSolver:
    def __init__(self, T_initial = 200, T_min = -np.inf, alpha = 1.1, max_steps=10000):
        self.steps = 0
        self.T_initial = T_initial
        self.T = T_initial
        self.T_min = T_min
        self.alpha = alpha
        self.max_steps = max_steps
        self.solution = None
        self.original_board = None
        
    def print(self):
        print(self.solution)
    
    def solve(self, board: SudokuBoard) -> Self:
        self.original_board = deepcopy(board)
        self.solution = deepcopy(board)
        if self.solution.is_solved():
            return self
        self._initial_fill()
        while True:
            self._simulated_annealing_step()
            self._temperature_schedule()
            
            if self.T < self.T_min or self.solution.is_solved() or self.steps > self.max_steps:
                self._print_info()
                return self

    def _print_info(self):
        print(f"Step: {self.steps}")
        print(f"Energy: {SudokuSolver.board_energy(self.solution)}")
        print(f"Temperature: {self.T}")
        rows_ok, cols_ok, squares_ok = self.solution.solution_status() 
        print(f"Rows rule ok: {rows_ok}")
        print(f"Cols rule ok: {cols_ok}")
        print(f"Squares rule ok: {squares_ok}")

    def _initial_fill(self):
        for square in range(9):
            rows = range(3 * int(square/3), 3 * (1 + int(square/3)))
            cols = range(3 * (square % 3), 3 * (1 + (square % 3)))
            for row in rows:
                for col in cols:
                    sub_board = self.solution.as_matrix()[rows, :][:, cols]
                    for number in set(list(range(1, 10))).difference(set(sub_board.A1)):
                        if self.solution.at(row, col) == 0:
                            self.solution.set(row, col, number)
                            break

    def _simulated_annealing_step(self):
        neighbor = self._generate_neighbor()
        delta_energy = SudokuSolver.board_energy(neighbor) - SudokuSolver.board_energy(self.solution)
        self.steps += 1
        if delta_energy < 0:
            self.solution = neighbor
            return
        else:
            r = np.random.uniform(0, 1, 1)
            if r < np.exp(-delta_energy/self.T):
                self.solution = neighbor
                return
    
    def _generate_neighbor(self):
        numbers = list(range(9))
        neighbor = deepcopy(self.solution)
        square = np.random.choice(numbers)
        rows = list(range(3 * int(square/3), 3 * (1 + int(square/3))))
        cols = list(range(3 * (square % 3), 3 * (1 + (square % 3))))
        row1, row2 = np.random.choice(rows, 2)
        col1, col2 = np.random.choice(cols, 2)
        while (row1, col1) == (row2, col2) or \
                self.original_board.at(row1, col1) or \
                    self.original_board.at(row2, col2):
            row1, row2 = np.random.choice(rows, 2)
            col1, col2 = np.random.choice(cols, 2)
        value1 = neighbor.at(row1, col1)
        value2 = neighbor.at(row2, col2)
        neighbor.set(row1, col1, value2)
        neighbor.set(row2, col2, value1)
        return neighbor
    
    def _temperature_schedule(self):
        self.T = self.alpha * self.T
        
    @staticmethod
    def board_energy(board: SudokuBoard):   
        matrix = board.as_matrix()
        
        rows_energies = np.apply_along_axis(
            SudokuSolver.row_or_col_score,
            axis=0,
            arr=np.asarray(matrix) 
        ).flatten()
        
        cols_energies = np.apply_along_axis(
            SudokuSolver.row_or_col_score,
            axis=1,
            arr=np.asarray(matrix) 
        ).flatten()

        return rows_energies.sum() + cols_energies.sum() + SudokuSolver.squares_score(matrix)
    
    @staticmethod
    def row_or_col_score(row_or_col):
        count = Counter(list(row_or_col))
        score = 0
        for value, freq in count.items():
            if value != 0 and freq == 1:
                score -= 1
        return score
    
    @staticmethod
    def squares_score(matrix):
        score = 0
        for square in range(9):
            rows = range(3 * int(square/3), 3 * (1 + int(square/3)))
            cols = range(3 * (square % 3), 3 * (1 + (square % 3)))
            sub_matrix = matrix[rows, :][:, cols]
            count = Counter(list(sub_matrix.A1))
            for value, freq in count.items():
                if value != 0 and freq == 1:
                    score -= 1
        return score

#+------+-------+------+
#|2 5   |     3 |   9 1|
#|  1   |     4 |      |
#|      |       |      |
#+------+-------+------+
#|      |       |      |
#|      |       |      |
#|      |       |      |
#+------+-------+------+
#|      |       |      |
#|      |       |      |
#|      |       |      |
#+------+-------+------+


In [30]:
board = SudokuBoard()

board.set_all([
    [0,0,2],[0,1,5],[0,5,3],[0,7,9],[0,8,1],
    [1,1,1],[1,5,4]
])

print(board)

agent = SudokuSolver(T_initial=0.5, T_min=0, alpha=0.99999, max_steps=np.inf)
start_time = time.time()
agent = agent.solve(board)
end_time = time.time()
final_board = agent.solution
print(f"Solution time: {end_time - start_time}")
print(f"Final Board:\n {final_board}")

Step: 2344
Energy: -243
Temperature: 0.48841623419005936
Rows rule ok: [ True  True  True  True  True  True  True  True  True]
Cols rule ok: [ True  True  True  True  True  True  True  True  True]
Squares rule ok: [ True  True  True  True  True  True  True  True  True]
Solution time: 3.1465566158294678
Final Board:
 #+------+-------+------+
#|2 5 4 | 6 8 3 | 7 9 1|
#|9 1 3 | 2 7 4 | 6 5 8|
#|6 7 8 | 1 9 5 | 3 2 4|
#+------+-------+------+
#|4 3 9 | 5 1 6 | 8 7 2|
#|1 8 5 | 9 2 7 | 4 3 6|
#|7 2 6 | 4 3 8 | 9 1 5|
#+------+-------+------+
#|5 4 7 | 3 6 1 | 2 8 9|
#|3 9 1 | 8 4 2 | 5 6 7|
#|8 6 2 | 7 5 9 | 1 4 3|
#+------+-------+------+


### Solução para outros 4 jogos

In [43]:
board = SudokuBoard()

board.set_all([
    [0,2,5],[0,1,3],[1,3,8],
    [3,1,1],[4,6,3],[4,0,4],
    [5,2,7],[6,1,2],[7,0,8]
])

print(board)

agent = SudokuSolver(T_initial=0.5, T_min=0, alpha=0.99999, max_steps=np.inf)
start_time = time.time()
agent = agent.solve(board)
end_time = time.time()
final_board = agent.solution
print(f"Solution time: {end_time - start_time}")
print(f"Final Board:\n {final_board}")

#+------+-------+------+
#|  3 5 |       |      |
#|      | 8     |      |
#|      |       |      |
#+------+-------+------+
#|  1   |       |      |
#|4     |       | 3    |
#|    7 |       |      |
#+------+-------+------+
#|  2   |       |      |
#|8     |       |      |
#|      |       |      |
#+------+-------+------+
Step: 4474
Energy: -243
Temperature: 0.4781229297902167
Rows rule ok: [ True  True  True  True  True  True  True  True  True]
Cols rule ok: [ True  True  True  True  True  True  True  True  True]
Squares rule ok: [ True  True  True  True  True  True  True  True  True]
Solution time: 5.859922885894775
Final Board:
 #+------+-------+------+
#|1 3 5 | 6 9 7 | 8 2 4|
#|2 7 6 | 8 3 4 | 9 1 5|
#|9 4 8 | 2 5 1 | 6 7 3|
#+------+-------+------+
#|5 1 2 | 3 6 9 | 7 4 8|
#|4 6 9 | 7 8 2 | 3 5 1|
#|3 8 7 | 4 1 5 | 2 6 9|
#+------+-------+------+
#|7 2 3 | 1 4 8 | 5 9 6|
#|8 5 4 | 9 2 6 | 1 3 7|
#|6 9 1 | 5 7 3 | 4 8 2|
#+------+-------+------+


In [35]:
board = SudokuBoard()

board.set_all([
    [0,0,2],[0,7,5],[0,3,3],[0,8,4],[0,6,1],
    [1,4,9],[3,5,2],[4,9,8],[5,7,3],[8,1,4]
])

print(board)

agent = SudokuSolver(T_initial=0.5, T_min=0, alpha=0.99999, max_steps=np.inf)
start_time = time.time()
agent = agent.solve(board)
end_time = time.time()
final_board = agent.solution
print(f"Solution time: {end_time - start_time}")
print(f"Final Board:\n {final_board}")

#+------+-------+------+
#|2     | 3     | 1 5 4|
#|      |   9   |      |
#|      |       |      |
#+------+-------+------+
#|      |     2 |      |
#|      |       |      |
#|      |       |   3  |
#+------+-------+------+
#|      |       |      |
#|      |       |      |
#|  4   |       |      |
#+------+-------+------+
Step: 3180
Energy: -243
Temperature: 0.4843500743714056
Rows rule ok: [ True  True  True  True  True  True  True  True  True]
Cols rule ok: [ True  True  True  True  True  True  True  True  True]
Squares rule ok: [ True  True  True  True  True  True  True  True  True]
Solution time: 4.157313823699951
Final Board:
 #+------+-------+------+
#|2 6 9 | 3 8 7 | 1 5 4|
#|1 5 4 | 2 9 6 | 3 8 7|
#|3 7 8 | 1 5 4 | 9 6 2|
#+------+-------+------+
#|7 9 1 | 8 3 2 | 5 4 6|
#|4 2 3 | 5 6 9 | 7 1 8|
#|6 8 5 | 7 4 1 | 2 3 9|
#+------+-------+------+
#|9 3 2 | 4 1 8 | 6 7 5|
#|8 1 7 | 6 2 5 | 4 9 3|
#|5 4 6 | 9 7 3 | 8 2 1|
#+------+-------+------+


In [37]:
board = SudokuBoard()

board.set_all([
    [0, 0, 5], [0, 1, 3], [0, 4, 7], 
    [1, 0, 6], [1, 3, 1], [1, 4, 9], [1, 5, 5], 
    [2, 1, 9], [2, 2, 8], [2, 7, 6], 
    [3, 0, 8], [3, 4, 6], [3, 8, 3], 
    [7, 3, 4], [7, 4, 1], [7, 5, 9], [7, 8, 5], 
    [8, 4, 8], [8, 7, 7], [8, 8, 9]
])


print(board)

agent = SudokuSolver(T_initial=0.5, T_min=0, alpha=0.99999, max_steps=np.inf)
start_time = time.time()
agent = agent.solve(board)
end_time = time.time()
final_board = agent.solution
print(f"Solution time: {end_time - start_time}")
print(f"Final Board:\n {final_board}")

#+------+-------+------+
#|5 3   |   7   |      |
#|6     | 1 9 5 |      |
#|  9 8 |       |   6  |
#+------+-------+------+
#|8     |   6   |     3|
#|      |       |      |
#|      |       |      |
#+------+-------+------+
#|      |       |      |
#|      | 4 1 9 |     5|
#|      |   8   |   7 9|
#+------+-------+------+
Step: 3868
Energy: -243
Temperature: 0.4810291662785046
Rows rule ok: [ True  True  True  True  True  True  True  True  True]
Cols rule ok: [ True  True  True  True  True  True  True  True  True]
Squares rule ok: [ True  True  True  True  True  True  True  True  True]
Solution time: 5.58279275894165
Final Board:
 #+------+-------+------+
#|5 3 4 | 8 7 6 | 1 9 2|
#|6 7 2 | 1 9 5 | 8 3 4|
#|1 9 8 | 3 2 4 | 5 6 7|
#+------+-------+------+
#|8 4 9 | 7 6 1 | 2 5 3|
#|2 5 6 | 9 4 3 | 7 8 1|
#|3 1 7 | 2 5 8 | 9 4 6|
#+------+-------+------+
#|9 2 5 | 6 3 7 | 4 1 8|
#|7 8 3 | 4 1 9 | 6 2 5|
#|4 6 1 | 5 8 2 | 3 7 9|
#+------+-------+------+


In [38]:
board = SudokuBoard()

board.set_all([
    [0, 2, 6], [0, 5, 8], [0, 7, 4],
    [1, 0, 5], [1, 3, 7], [1, 5, 9], [1, 6, 6], 
    [2, 1, 8], [2, 4, 4], [2, 6, 7], [2, 8, 2],
    [3, 2, 1], [3, 5, 5], [7, 3, 2], [7, 5, 6], 
    [7, 8, 8], [8, 1, 7], [8, 3, 3], [8, 6, 5]
])

print(board)

agent = SudokuSolver(T_initial=0.5, T_min=0, alpha=0.99999, max_steps=np.inf)
start_time = time.time()
agent = agent.solve(board)
end_time = time.time()
final_board = agent.solution
print(f"Solution time: {end_time - start_time}")
print(f"Final Board:\n {final_board}")

#+------+-------+------+
#|    6 |     8 |   4  |
#|5     | 7   9 | 6    |
#|  8   |   4   | 7   2|
#+------+-------+------+
#|    1 |     5 |      |
#|      |       |      |
#|      |       |      |
#+------+-------+------+
#|      |       |      |
#|      | 2   6 |     8|
#|  7   | 3     | 5    |
#+------+-------+------+
Step: 3906
Energy: -243
Temperature: 0.48084640900761233
Rows rule ok: [ True  True  True  True  True  True  True  True  True]
Cols rule ok: [ True  True  True  True  True  True  True  True  True]
Squares rule ok: [ True  True  True  True  True  True  True  True  True]
Solution time: 5.547661304473877
Final Board:
 #+------+-------+------+
#|7 2 6 | 5 1 8 | 3 4 9|
#|5 3 4 | 7 2 9 | 6 8 1|
#|1 8 9 | 6 4 3 | 7 5 2|
#+------+-------+------+
#|9 6 1 | 4 8 5 | 2 3 7|
#|3 5 7 | 1 6 2 | 8 9 4|
#|2 4 8 | 9 3 7 | 1 6 5|
#+------+-------+------+
#|6 9 5 | 8 7 1 | 4 2 3|
#|4 1 3 | 2 5 6 | 9 7 8|
#|8 7 2 | 3 9 4 | 5 1 6|
#+------+-------+------+
